In [2]:
import os
import boto3
import time

'''
    Set up the python query method
'''
cluster_client = boto3.client('redshift', aws_access_key_id=os.getenv('aws_access_key'),
    aws_secret_access_key=os.getenv('aws_secret_key'), region_name = 'us-west-2')
my_cluster = cluster_client.describe_clusters()['Clusters'][0]

data_client = boto3.client('redshift-data', aws_access_key_id=os.getenv('aws_access_key'),
    aws_secret_access_key=os.getenv('aws_secret_key'), region_name = 'us-west-2')

'''
    Designed queries to look into Redshift
'''
def execute_redshift_query(sql):
    print(sql)
    output = data_client.execute_statement(
                    ClusterIdentifier=my_cluster['ClusterIdentifier'],
                    Database='ohmconnect',
                    DbUser='gregory',
                    Sql=sql)
    status = 'SUBMITTED'
    cnt = 2
    time.sleep(min(cnt, 10))
    while cnt < 1000:
        result = data_client.describe_statement(Id=output['Id'])
        status = result['Status']
        if status in {'FAILED', 'ABORTED'}:
            print(result['Error'])
            print(status)
            return (output['Id'], status)
        elif status in {'FINISHED'}:
            print(status)
            return (output['Id'], status)
        else:
            print('Query status: ' + status)
            print(f'Will sleep for {min(cnt, 10)}s \n')
            time.sleep(min(cnt, 10))
            cnt += 1
    print('Query has been running for too long, result will not be checked anymore.')
    print('Last status: ' + status)
    return (output['Id'], status)

'''
    Return answer from Redshift after running the select query. 
'''
def get_query_result(sql):
    output_id, status = execute_redshift_query(sql)
    if status != 'FINISHED':
        return 'Error'
    response = data_client.get_statement_result(Id=output_id)
    column_list = [item['label'] for item in response['ColumnMetadata']]
    records = response['Records']
    while 'NextToken' in response:
        response = data_client.get_statement_result(Id=output_id, NextToken = response['NextToken'])
        records += response['Records']
    
    output_arr = []
    for row in records:
        new_row = [list(item.values())[0] for item in row]
        output_arr.append(new_row)
    
    return (output_arr, column_list)

In [219]:
import pandas as pd
'''
    Query that returns top performers by month in a csv file. 
'''
query = '''
SELECT userid, month, points_per_month, Rank
FROM
(
    SELECT userid
    , month
    , points_per_month
    , rank() OVER (partition by month order by points_per_month DESC) as Rank
    FROM
    (
        SELECT userid, DATE_PART('month', "date") as month, SUM(points) as points_per_month
        FROM ohm_connect.event_performance
        where date is not NULL  -- error data was kept in the load. Eliminated here.  
        GROUP BY 1,2
    )
)
WHERE Rank < 3
'''
records, column_list = get_query_result(query)
pd.DataFrame.from_records(records, columns = column_list)
frame.to_csv('top_two_performers.csv')


SELECT userid, month, points_per_month, Rank
FROM
(
    SELECT userid
    , month
    , points_per_month
    , rank() OVER (partition by month order by points_per_month DESC) as Rank
    FROM
    (
        SELECT userid, DATE_PART('month', "date") as month, SUM(points) as points_per_month
        FROM ohm_connect.event_performance
        where date is not NULL  -- error data was kept in the load. Eliminated here.  
        GROUP BY 1,2
    )
)
WHERE Rank < 3

FINISHED


In [34]:
import pandas as pd
#data summary
query = '''
SELECT attribute2
, SUM(sum_of_points) as sum_of_points
, AVG(sum_of_points) as avg_points_per_user
, SUM(cnts) as total_cnts
FROM
(
    SELECT userid, SUM(points) as sum_of_points, COUNT(userid) as cnts
    FROM ohm_connect.event_performance
    where date is not NULL  -- error data was kept in the load. Eliminated here.  
    GROUP BY 1
) as erf
JOIN ohm_connect.users as usr ON usr.userid = erf.userid
GROUP BY 1
'''
records, column_list = get_query_result(query)
pd.DataFrame.from_records(records, columns = column_list)


SELECT attribute2
, SUM(sum_of_points) as sum_of_points
, AVG(sum_of_points) as avg_points_per_user
, SUM(cnts) as total_cnts
FROM
(
    SELECT userid, SUM(points) as sum_of_points, COUNT(userid) as cnts
    FROM ohm_connect.event_performance
    where date is not NULL  -- error data was kept in the load. Eliminated here.  
    GROUP BY 1
) as erf
JOIN ohm_connect.users as usr ON usr.userid = erf.userid
GROUP BY 1

FINISHED


,attribute2,sum_of_points,avg_points_per_user,total_cnts
0,A,548612,1627,12172
1,B,993662,2405,16581
2,C,5689448,25861,8400


In [6]:
import pandas as pd
#data summary
query = '''
SELECT erf.userid, month, points_per_month, high_rank, low_rank, point_cnt
, attribute1, attribute2
FROM
(
    SELECT userid
    , month
    , points_per_month
    , rank() OVER (partition by month order by points_per_month DESC) as high_rank
    , rank() OVER (partition by month order by points_per_month ASC) as low_rank
    , cnt as point_cnt
    FROM
    (
        SELECT userid, DATE_PART('month', "date") as month, SUM(points) as points_per_month, COUNT(points) as cnt
        FROM ohm_connect.event_performance
        where date is not NULL  -- error data was kept in the load. Eliminated here.  
        GROUP BY 1,2
    )
) as erf
JOIN ohm_connect.users as usr ON usr.userid = erf.userid
WHERE high_rank < 5 or low_rank < 5
'''
records, column_list = get_query_result(query)
frame = pd.DataFrame.from_records(records, columns = column_list)
frame.to_csv('output.csv')
frame


SELECT erf.userid, month, points_per_month, high_rank, low_rank, point_cnt
, attribute1, attribute2
FROM
(
    SELECT userid
    , month
    , points_per_month
    , rank() OVER (partition by month order by points_per_month DESC) as high_rank
    , rank() OVER (partition by month order by points_per_month ASC) as low_rank
    , cnt as point_cnt
    FROM
    (
        SELECT userid, DATE_PART('month', "date") as month, SUM(points) as points_per_month, COUNT(points) as cnt
        FROM ohm_connect.event_performance
        where date is not NULL  -- error data was kept in the load. Eliminated here.  
        GROUP BY 1,2
    )
) as erf
JOIN ohm_connect.users as usr ON usr.userid = erf.userid
WHERE high_rank < 5 or low_rank < 5

Query status: STARTED
Will sleep for 2s 

FINISHED


,userid,month,points_per_month,high_rank,low_rank,point_cnt,attribute1,attribute2
0,8009bd3a-2c2c-465e-b7cb-7e051d399572,1.0,-6995,441,1,1,0,B
1,df26b16b-41e6-414f-8346-36ced8fdcd37,1.0,-5560,440,2,1,0,C
2,beb9aef6-0ab5-4718-b2f1-199963fd542e,1.0,-4699,439,3,1,0,A
3,874a081f-0676-4319-9219-4c864ac5a401,1.0,-4376,438,4,1,0,A
4,c2926ad9-f9c8-4e32-a792-dd0d2b082bde,1.0,3054,4,438,2,0,C
...,...,...,...,...,...,...,...,...
91,69720b3e-07a2-4d37-9c11-467609543eaf,12.0,-3176,826,4,3,0,B
92,424fc967-5e61-4e77-afd8-619b03cec9e4,12.0,5225,4,826,3,1,C
93,d272e8a1-b683-48a5-979f-adf8f97766fe,12.0,5270,3,827,3,0,C
94,83b47be7-8b84-481c-a6f8-de817f6a8db8,12.0,5542,2,828,3,1,C


In [14]:
import pandas as pd
#data summary
query = '''
SELECT 
CASE 
    WHEN day_of_week = 0 THEN 'Sunday'
    WHEN day_of_week = 1 THEN 'Monday'
    WHEN day_of_week = 2 THEN 'Tuesday'
    WHEN day_of_week = 3 THEN 'Wednesday'
    WHEN day_of_week = 4 THEN 'Thursday'
    WHEN day_of_week = 5 THEN 'Friday'
    ELSE 'Saturday'
END as day_of_week
, SUM(points_per_day)
, AVG(points_per_day) as avg_points_per_day
, SUM(cnts) as total_cnts
FROM
(
    SELECT userid, DATE_PART('dayofweek', "date") as day_of_week, SUM(points) as points_per_day
    , COUNT(points) as cnts 
    FROM ohm_connect.event_performance
    where date is not NULL  -- error data was kept in the load. Eliminated here. 
    GROUP BY 1,2
)
GROUP BY 1
order by 2
'''
records, column_list = get_query_result(query)
frame = pd.DataFrame.from_records(records, columns = column_list)
frame.to_csv('output.csv')
frame


SELECT 
CASE 
    WHEN day_of_week = 0 THEN 'Sunday'
    WHEN day_of_week = 1 THEN 'Monday'
    WHEN day_of_week = 2 THEN 'Tuesday'
    WHEN day_of_week = 3 THEN 'Wednesday'
    WHEN day_of_week = 4 THEN 'Thursday'
    WHEN day_of_week = 5 THEN 'Friday'
    ELSE 'Saturday'
END as day_of_week
, SUM(points_per_day)
, AVG(points_per_day) as avg_points_per_day
, SUM(cnts) as total_cnts
FROM
(
    SELECT userid, DATE_PART('dayofweek', "date") as day_of_week, SUM(points) as points_per_day
    , COUNT(points) as cnts 
    FROM ohm_connect.event_performance
    where date is not NULL  -- error data was kept in the load. Eliminated here. 
    GROUP BY 1,2
)
GROUP BY 1
order by 2

FINISHED


,day_of_week,sum,avg_points_per_day,total_cnts
0,Saturday,334797,452,1348
1,Sunday,421308,555,1894
2,Tuesday,550126,597,5944
3,Thursday,782878,850,4001
4,Monday,1031284,1091,6928
5,Wednesday,1769736,1884,7691
6,Friday,2371339,2541,9764


In [236]:
import pandas as pd
'''
    Analysis of the average points per day


'''
query = '''
SELECT 
day
, SUM(points_per_day)
, AVG(points_per_day) as avg_points_per_day
, SUM(cnts) as total_cnts
FROM
(
    SELECT userid, DATE_PART('day', "date") as day, SUM(points) as points_per_day
    , COUNT(points) as cnts 
    FROM ohm_connect.event_performance
    where date is not NULL  -- error data was kept in the load. Eliminated here.  
    GROUP BY 1,2
)
GROUP BY 1
order by 2
'''
records, column_list = get_query_result(query)
frame = pd.DataFrame.from_records(records, columns = column_list)
frame.to_csv('output.csv')
frame


SELECT 
day
, SUM(points_per_day)
, AVG(points_per_day) as avg_points_per_day
, SUM(cnts) as total_cnts
FROM
(
    SELECT userid, DATE_PART('day', "date") as day, SUM(points) as points_per_day
    , COUNT(points) as cnts 
    FROM ohm_connect.event_performance
    where date is not NULL  -- error data was kept in the load. Eliminated here.  
    GROUP BY 1,2
)
GROUP BY 1
order by 2

FINISHED


,day,sum,avg_points_per_day,total_cnts
0,11.0,-208098,-337,1394
1,29.0,21718,75,286
2,7.0,36370,101,391
3,9.0,66660,131,538
4,16.0,82408,206,551
5,12.0,106469,166,861
6,10.0,115282,156,1527
7,3.0,121203,214,593
8,27.0,134398,264,610
9,20.0,147560,231,843


In [33]:
import pandas as pd




query = '''

with point_rank as
(
    SELECT userid
    , month
    , points_per_month
    , rank() OVER (partition by month order by points_per_month DESC) as high_rank
    FROM
    (
        SELECT userid, DATE_PART('month', "date") as month, SUM(points) as points_per_month
        FROM ohm_connect.event_performance
        where date is not NULL  -- error data was kept in the load. Eliminated here. 
        GROUP BY 1,2
    )
)

, user_month as
(    
    SELECT erf.userid, erf.month
    FROM point_rank as erf
    WHERE high_rank < 50
)

, doubling as
(
    SELECT DISTINCT u1.userid
    FROM user_month as u1
    JOIN user_month as u2 ON u1.userid = u2.userid
    and u1.month != u2.month
)




SELECT month, attribute1, COUNT(DISTINCT erf.userid) as cnt_top_repeat_users, SUM(points_per_month) as sum_points
--, points_per_month, high_rank
--, attribute1, attribute2
FROM point_rank as erf
JOIN ohm_connect.users as usr ON usr.userid = erf.userid
WHERE high_rank < 50
and erf.userid IN (SELECT userid FROM doubling)
and attribute1 = 1
GROUP BY 1, 2
order by month

'''
records, column_list = get_query_result(query)
frame = pd.DataFrame.from_records(records, columns = column_list)
frame.to_csv('output.csv')
frame
    



with point_rank as
(
    SELECT userid
    , month
    , points_per_month
    , rank() OVER (partition by month order by points_per_month DESC) as high_rank
    FROM
    (
        SELECT userid, DATE_PART('month', "date") as month, SUM(points) as points_per_month
        FROM ohm_connect.event_performance
        where date is not NULL  -- error data was kept in the load. Eliminated here. 
        GROUP BY 1,2
    )
)

, user_month as
(    
    SELECT erf.userid, erf.month
    FROM point_rank as erf
    WHERE high_rank < 50
)

, doubling as
(
    SELECT DISTINCT u1.userid
    FROM user_month as u1
    JOIN user_month as u2 ON u1.userid = u2.userid
    and u1.month != u2.month
)




SELECT month, attribute1, COUNT(DISTINCT erf.userid) as cnt_top_repeat_users, SUM(points_per_month) as sum_points
--, points_per_month, high_rank
--, attribute1, attribute2
FROM point_rank as erf
JOIN ohm_connect.users as usr ON usr.userid = erf.userid
WHERE high_rank < 50
and erf.userid IN (SELECT userid 

,month,attribute1,cnt_top_repeat_users,sum_points
0,1.0,1,6,13647
1,2.0,1,13,86686
2,3.0,1,12,54703
3,4.0,1,9,57851
4,5.0,1,12,79924
5,6.0,1,11,163185
6,7.0,1,18,178684
7,8.0,1,16,417400
8,9.0,1,15,225815
9,10.0,1,15,65508


In [57]:
import numpy as np
query = '''
SELECT userid
, attribute1
, attribute2
, sum_of_points
FROM
(
    SELECT erf.userid
    , attribute1
    , CASE 
        WHEN attribute2 = 'A' THEN 1
        WHEN attribute2 = 'B' THEN 2
        ELSE 3
    END as attribute2
    , SUM(points) as sum_of_points
    FROM ohm_connect.event_performance as erf
    JOIN ohm_connect.users as u2 ON u2.userid = erf.userid
    where date is not NULL  -- error data was kept in the load. Eliminated here.  
    GROUP BY 1,2,3
)
'''
#data_frame = pd.DataFrame.from_records(records, columns = column_list).set_index('userid')
records, column_list = get_query_result(query)
#data_frame = pd.DataFrame.from_records(records, columns = column_list).set_index('userid')
df = pd.DataFrame.from_records(records, columns = column_list)
print(df['sum_of_points'].corr(df['attribute1']))
print(df['sum_of_points'].corr(df['attribute2']))
print(df['attribute1'].corr(df['attribute2']))




SELECT userid
, attribute1
, attribute2
, sum_of_points
FROM
(
    SELECT erf.userid
    , attribute1
    , CASE 
        WHEN attribute2 = 'A' THEN 1
        WHEN attribute2 = 'B' THEN 2
        ELSE 3
    END as attribute2
    , SUM(points) as sum_of_points
    FROM ohm_connect.event_performance as erf
    JOIN ohm_connect.users as u2 ON u2.userid = erf.userid
    where date is not NULL  -- error data was kept in the load. Eliminated here.  
    GROUP BY 1,2,3
)

FINISHED
0.17020535465077305
0.4721156902158399
0.02547363033022271


In [ ]:
0.17020535465077305
0.4721156902158399
#all together

0.1333023571287215
0.5381206416860103
#positive

0.04886588148313559
-0.2706691255642229
0.025773006563522903
#negative